In [2]:
import sys
sys.path.append('/mnt/disk05/note/workspaces/duxx')
from feature_selector import FeatureSelector
from unity import init_pkg as ini
from unity.eda_func import EdaFunc as edf
from unity.fea_eda_func import FeaEdaFunc as fef
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve
import os
import json
import datetime
import gc
import scorecardpy as sc
# import pandas_profiling

% matplotlib inline

In [3]:
datset = ini.pd.read_csv('data/comb_main.csv')
datset = datset.assign(gender = lambda x:(x.cert_no.str.get(16).astype(int) % 2 == 0).astype(int),\
              resid_prov = lambda x:x.cert_no.str.slice(0,7).astype(int) // 100000 * 10000)
bdmain = datset[datset['type']==3]

In [4]:
bdmain.head()

,cert_no,LN_CNT_UNSQUARE_LEVEL,CC_LIMIT_MAX_LEVEL,CC_ORG_CNT_LEVEL,qd,CC_DELQ_NOW_OVDUE_MAX_LEVEL,CC_DELQ_NOW_STAT_MAX_LEVEL,CC_LN_QRY_TIMES_L1ST_LEVEL,HOUSE_FUND_MONTH_PAYAMT,tag,CUSTOMER_NO,mobile_no,cust_name(md5),mobile_no(md5),zx_cc_now_total_amt_level,zx_ln_now_total_amt_level,zx_cc_use_rate,type,gender,resid_prov
1463,450702199312264810,1002.0,1002.0,1003.0,mt,1002,1003,1002,NaN,0.0,450702199312264810,17707773277,fea65a61c0f6416dd772edbd1d2fc716,3bd2267657accc83eda252e720a7eee7,1.0,1.0,1.0,3,0,450000
13177,422201199206072236,1003.0,1002.0,1003.0,mt,1002,1003,1002,NaN,0.0,422201199206072236,15549638496,f98cafeeafe2ce4e02fff52ba104b709,9fa8bfc2c5f4983287c2614a05a11e47,1.0,1.0,1.0,3,0,420000
15558,412728199010114030,1003.0,1002.0,1003.0,mt,1002,1002,1002,NaN,0.0,412728199010114030,18638062671,72ebfc1389c369fce37054b459ca400c,5a5e8cd3849aa97e76d4e0cb076ebdd4,1.0,1.0,1.0,3,0,410000
22105,350426199609014522,1002.0,1002.0,1003.0,mt,1002,1003,1002,NaN,0.0,350426199609014522,18659438096,961f561b314c032c3fed940ecb761358,09cecaa4730ba312527bf63fae837359,1.0,1.0,1.0,3,1,350000
26086,520103197712202013,1002.0,1003.0,1003.0,mt,1002,1002,1002,NaN,0.0,520103197712202013,18198235828,1e49b429ec5ed48dfef0be87ebcef500,5c448515bc1049e8aa7cead0e425eaa6,1.0,1.0,0.0,3,0,520000


In [5]:
train_x = bdmain.loc[:,~bdmain.columns.str.contains('cert_no|type|tag|md5|CUS|mobile_no|qd|PAYAMT')]
train_y = bdmain['tag']

In [6]:
train_x.isnull().any()

LN_CNT_UNSQUARE_LEVEL           True
CC_LIMIT_MAX_LEVEL              True
CC_ORG_CNT_LEVEL                True
CC_DELQ_NOW_OVDUE_MAX_LEVEL    False
CC_DELQ_NOW_STAT_MAX_LEVEL     False
CC_LN_QRY_TIMES_L1ST_LEVEL     False
zx_cc_now_total_amt_level       True
zx_ln_now_total_amt_level       True
zx_cc_use_rate                 False
gender                         False
resid_prov                     False
dtype: bool

In [7]:
train_x = train_x.drop('CC_LN_QRY_TIMES_L1ST_LEVEL',axis=1)

In [8]:
train_x.loc[:,['zx_cc_now_total_amt_level','zx_ln_now_total_amt_level']] = train_x.fillna(-99)
train_x.loc[:,['LN_CNT_UNSQUARE_LEVEL','CC_LIMIT_MAX_LEVEL','CC_ORG_CNT_LEVEL']] = train_x.fillna(1001)

In [9]:
train_x.sum()/10

LN_CNT_UNSQUARE_LEVEL             501547.4
CC_LIMIT_MAX_LEVEL                501327.4
CC_ORG_CNT_LEVEL                  502878.4
CC_DELQ_NOW_OVDUE_MAX_LEVEL       501006.3
CC_DELQ_NOW_STAT_MAX_LEVEL        500509.8
zx_cc_now_total_amt_level          -3520.0
zx_ln_now_total_amt_level            430.0
zx_cc_use_rate                     -4560.9
gender                               132.4
resid_prov                     186840000.0
dtype: float64

In [10]:
res = LogisticRegression(class_weight='balanced')
res.fit(train_x,train_y)

LogisticRegression(class_weight='balanced')

In [11]:
# x['_const'] = 1
predict_train_y = res.predict(train_x)
roc_auc_score(train_y,predict_train_y)

0.5260642156013844